In [7]:
import pandas as pd
import numpy as np 
import glob 
import warnings 
from collections import Counter
warnings.filterwarnings("ignore")
import plotly.express as px 
import seaborn as sns 
import matplotlib.pyplot as plt 
import lightgbm as lgbm 
from sklearn.model_selection import StratifiedKFold, train_test_split
import math
import os 
import random
import torch 
import pyarrow as pa
import pyarrow.parquet as pq
import torch.nn as nn
#from transformers import AdamW
from torch.utils.data import Dataset , DataLoader
from colorama import Fore , Style
r__=Fore.RED
g__=Fore.GREEN
st__=Style.RESET_ALL

In [8]:
r__=Fore.RED
g__=Fore.GREEN
st__=Style.RESET_ALL
def wap(row) :
    denom = row.ask_size1 + row.bid_size1
    return ((row.bid_price1 * row.ask_size1 + row.ask_price1 * row.bid_size1)/denom)
def log_return(list_prices):
    return np.log(list_prices).diff()
def realized_volatility(series_log_return):
    return np.sqrt(np.sum(series_log_return**2))
def custom_loss(ytrue,ypred) :
    squared_residual = (ytrue-ypred)**2/ytrue
    grad = squared_residual
    hess = np.ones(len(ytrue))
    
    return grad,hess
def rmspe(y_true, y_pred):
    return  (np.sqrt(np.mean(np.square((y_true - y_pred) / y_true))))
def feval_RMSPE(preds, train_data):
    labels = train_data.get_label()
    return 'RMSPE', round(rmspe(y_true = labels, y_pred = preds),5), False
def custom_rmspe_valid(y_true, y_pred):
    residual = (y_true - y_pred).astype("float")
    residual = residual ** 2 / y_true
    residual = np.mean(residual)
    return "eval_RMSPE", math.sqrt(residual), False
def simple_volatility(series_prix):
    mx = np.max(series_prix)
    mn = np.min(series_prix)
    moy = np.mean(series_prix)
    vol = (moy-mn)/(mx-mn)
    return vol
def count_unique(series):
    return len(np.unique(series))

In [9]:
train = pd.read_parquet("target_data/target_train.parquet")
train

,stock_id,time_id,target
0,0,5,0.004136
1,0,11,0.001445
2,0,16,0.002168
3,0,31,0.002195
4,0,62,0.001747
...,...,...,...
428162,126,25653,0.003977
428163,126,25654,0.001674
428164,126,25668,0.006922
428165,126,25680,0.002081


In [23]:
book_train_stock_id_0 = pd.read_parquet("stock_train/stock_0_train.parquet")
book_train_stock_id_0_t_id5 = book_train_stock_id_0[book_train_stock_id_0["time_id"]==5]
book_train_stock_id_0_t_id5

,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2
0,5,0,1.001422,1.002301,1.001370,1.002353,3,226,2,100
1,5,1,1.001422,1.002301,1.001370,1.002353,3,100,2,100
2,5,5,1.001422,1.002301,1.001370,1.002405,3,100,2,100
3,5,6,1.001422,1.002301,1.001370,1.002405,3,126,2,100
4,5,7,1.001422,1.002301,1.001370,1.002405,3,126,2,100
...,...,...,...,...,...,...,...,...,...,...
297,5,585,1.003129,1.003749,1.003025,1.003801,100,3,26,3
298,5,586,1.003129,1.003749,1.002612,1.003801,100,3,2,3
299,5,587,1.003129,1.003749,1.003025,1.003801,100,3,26,3
300,5,588,1.003129,1.003749,1.002612,1.003801,100,3,2,3


In [34]:
trade_train_stock_id_0 = pd.read_parquet("stock_trade_train/stock_0_train.parquet")

In [45]:
trade_train_stock_id_0 

,time_id,seconds_in_bucket,price,size,order_count
0,5,21,1.002301,326,12
1,5,46,1.002778,128,4
2,5,50,1.002818,55,1
3,5,57,1.003155,121,5
4,5,68,1.003646,4,1
...,...,...,...,...,...
99253,25683,473,1.002081,106,5
99254,25683,512,1.001077,100,3
99255,25683,529,1.001027,2,1
99256,25683,566,0.999374,2,2


In [ ]:
###EXPLORING POSSIBLE FEATURES###

In [75]:
#Feature 1 -> Relative volume by 10min bucket

#1. Finding average size (volume) across all time_id's

avg_trade_volume_stock_id_0 = trade_train_stock_id_0.groupby(['time_id']).sum()["size"].mean() #this groups by time id, gets the total sum of the size, then it gets the mean across all time buckets 
 
#2. Get size (volume) of time_id = 5
trade_volume_stock_id_0_time_id_5 = trade_train_stock_id_0[trade_train_stock_id_0['time_id'] == 5]["size"].sum() #gives me the volume of time bucket 5

#3. Compute relative volume
rel_trade_volume_stock_id_0_time_id_5 = trade_volume_stock_id_0_time_id_5/avg_trade_volume_stock_id_0
rel_trade_volume_stock_id_0_time_id_5

0.9872250681883579

In [104]:
#Feature 2 -> Relative price range 

#1. Find min and max of trade price for all seperate time_id. 
min_trade_price_stock_id_0 = trade_train_stock_id_0.groupby(['time_id']).min()
max_trade_price_stock_id_0 = trade_train_stock_id_0.groupby(['time_id']).max()
#2. Find range of trade price and median of trade price for all seperate time_id. 
range_trade_price_stock_id_0 = max_trade_price_stock_id_0 - min_trade_price_stock_id_0 
median_trade_price_stock_id_0 = trade_train_stock_id_0.groupby(['time_id']).median()
#3. Use median to compute how much percent below our minimum is for all seperate time_id. 
lower_percent_range_relative_to_median = (median_trade_price_stock_id_0 - min_trade_price_stock_id_0)/median_trade_price_stock_id_0
#4. Use median to compute how much percent above our maximum is for all seperate time_id. 
upper_percent_range_relative_to_median = (max_trade_price_stock_id_0 - median_trade_price_stock_id_0)/median_trade_price_stock_id_0
#5 Add both values to get total percent range. E.g. 3% below median and 5% above median = 8% total range for all seperate time_id.
total_percent_range = upper_percent_range_relative_to_median + lower_percent_range_relative_to_median

#6. Compute the average percent range across all time id's
avg_total_percent_range = total_percent_range["price"].mean()

#7. Get total percent range for time id 5
min_price_stock_id_0_time_id_5 = trade_train_stock_id_0[trade_train_stock_id_0['time_id'] == 5]["price"].min()
max_price_stock_id_0_time_id_5 = trade_train_stock_id_0[trade_train_stock_id_0['time_id'] == 5]["price"].max()

range_stock_id_0_time_id_5 = max_price_stock_id_0_time_id_5 - min_price_stock_id_0_time_id_5 #gives me the range of price for time bucket 5
median_stock_id_0_time_id_5 = trade_train_stock_id_0[trade_train_stock_id_0['time_id'] == 5]["price"].median()

lower_percent_range_relative_to_median_stock_id_0_time_id_5 = (median_stock_id_0_time_id_5 - min_price_stock_id_0_time_id_5)/median_stock_id_0_time_id_5

upper_percent_range_relative_to_median_stock_id_0_time_id_5= (max_price_stock_id_0_time_id_5 - median_stock_id_0_time_id_5)/median_stock_id_0_time_id_5

total_percent_range_stock_id_0_time_id_5 = lower_percent_range_relative_to_median_stock_id_0_time_id_5 + upper_percent_range_relative_to_median_stock_id_0_time_id_5

#8. Compute relative percent trading range to average
rel_total_percent_range_stock_id_0_time_id_5 = total_percent_range_stock_id_0_time_id_5/avg_total_percent_range
rel_total_percent_range_stock_id_0_time_id_5

0.70972

In [ ]:
#Feature 3 -> Time between execution --> size/second 